<a href="https://colab.research.google.com/github/ericatsu/msc_capsmodel/blob/main/caps_mri.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Libraries

In [6]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
from google.colab import drive
from collections import Counter
from sklearn.utils.class_weight import compute_class_weight

## Dateset and Parameters

In [7]:
drive.mount('/content/drive', force_remount=True)

base_dir = "/content/drive/MyDrive/msc_capsmodel/dataset"
test_dir = os.path.join(base_dir, 'testing')
train_dir = os.path.join(base_dir, 'training')
categories = ['glioma', 'meningioma', 'notumor', 'pituitary']

img_size = 128
batch_size = 16
epochs = 50

Mounted at /content/drive


## Data Preparation

In [8]:
def prepare_data(directory, categories, img_size):
    data = []
    labels = []
    for category in categories:
        path = os.path.join(directory, category)
        class_num = categories.index(category)
        for img in os.listdir(path):
            try:
                img_array = tf.keras.preprocessing.image.load_img(os.path.join(path, img), target_size=(img_size, img_size), color_mode="grayscale")
                img_array = tf.keras.preprocessing.image.img_to_array(img_array)
                img_array = img_array / 255.0
                data.append(img_array)
                labels.append(class_num)
            except Exception as e:
                print(f"Failed to read image: {os.path.join(path, img)}, error: {e}")  # Debug statement
    return np.array(data), np.array(labels)

# Load and preprocess data
X_train, y_train = prepare_data(train_dir, categories, img_size)
X_test, y_test = prepare_data(test_dir, categories, img_size)
y_train = to_categorical(y_train, num_classes=len(categories))
y_test = to_categorical(y_test, num_classes=len(categories))

In [9]:
print(f"Training samples: {X_train.shape[0]}")
print(f"Testing samples: {X_test.shape[0]}")

Training samples: 5711
Testing samples: 1311


In [10]:
print(f"Training class distribution: {Counter(np.argmax(y_train, axis=1))}")
print(f"Testing class distribution: {Counter(np.argmax(y_test, axis=1))}")

Training class distribution: Counter({2: 1595, 3: 1457, 1: 1339, 0: 1320})
Testing class distribution: Counter({2: 405, 1: 306, 0: 300, 3: 300})


### Data Augmentation

In [11]:
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)
train_generator = datagen.flow(X_train, y_train, batch_size=batch_size)


### Capsule Network Model

In [12]:
class CapsuleLayer(layers.Layer):
    def __init__(self, num_capsules, dim_capsules, routings=3, **kwargs):
        super(CapsuleLayer, self).__init__(**kwargs)
        self.num_capsules = num_capsules
        self.dim_capsules = dim_capsules
        self.routings = routings
        self.kernel = None

    def build(self, input_shape):
        assert len(input_shape) == 3, "Input shape should be [None, input_capsule_num, input_capsule_dim]"
        self.input_capsule_num = input_shape[1]
        self.input_capsule_dim = input_shape[2]
        self.kernel = self.add_weight(shape=(self.input_capsule_num, self.num_capsules, self.input_capsule_dim, self.dim_capsules),
                                      initializer='glorot_uniform',
                                      name='capsule_kernel')

    def call(self, inputs, training=None):
        inputs_expand = tf.expand_dims(tf.expand_dims(inputs, 2), -1)
        inputs_tiled = tf.tile(inputs_expand, [1, 1, self.num_capsules, 1, 1])
        inputs_hat = tf.reduce_sum(inputs_tiled * self.kernel, axis=3)

        b = tf.zeros(shape=[tf.shape(inputs)[0], self.input_capsule_num, self.num_capsules, 1])
        for i in range(self.routings):
            c = tf.nn.softmax(b, axis=2)
            outputs = tf.reduce_sum(c * inputs_hat, axis=1, keepdims=True)
            outputs = tf.squeeze(tf.nn.l2_normalize(outputs, -1), axis=1)
            if i < self.routings - 1:
                b += tf.reduce_sum(inputs_hat * tf.expand_dims(outputs, 1), -1, keepdims=True)

        return outputs

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.num_capsules, self.dim_capsules)

class Length(layers.Layer):
    def call(self, inputs, **kwargs):
        return tf.sqrt(tf.reduce_sum(tf.square(inputs), -1) + 1e-9)

### Define the CapsNet model

In [13]:
input_image = layers.Input(shape=(img_size, img_size, 1))
conv1 = layers.Conv2D(256, (9, 9), activation='relu')(input_image)
conv2 = layers.Conv2D(256, (9, 9), strides=(2, 2), activation='relu')(conv1)
primary_caps = layers.Conv2D(32 * 8, (9, 9), activation='relu')(conv2)

print("Shape of primary_caps:", primary_caps.shape)

Shape of primary_caps: (None, 48, 48, 256)


### Adjust reshape layer based on printed shape

In [14]:
primary_caps_reshape = layers.Reshape((-1, 256))(primary_caps)
capsule = CapsuleLayer(num_capsules=len(categories), dim_capsules=16)(primary_caps_reshape)
output = Length()(capsule)

model = models.Model(inputs=input_image, outputs=output)
model.compile(optimizer=optimizers.Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 128, 1)]     0         
                                                                 
 conv2d (Conv2D)             (None, 120, 120, 256)     20992     
                                                                 
 conv2d_1 (Conv2D)           (None, 56, 56, 256)       5308672   
                                                                 
 conv2d_2 (Conv2D)           (None, 48, 48, 256)       5308672   
                                                                 
 reshape (Reshape)           (None, 2304, 256)         0         
                                                                 
 capsule_layer (CapsuleLaye  (None, 4, 16)             37748736  
 r)                                                              
                                                             

In [15]:
class_weights = compute_class_weight('balanced', classes=np.unique(np.argmax(y_train, axis=1)), y=np.argmax(y_train, axis=1))
class_weights = dict(enumerate(class_weights))

## Training the Model

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=len(X_train) // batch_size,
    epochs=epochs,
    validation_data=(X_test, y_test),
    class_weight=class_weights
)

Epoch 1/50
 15/356 [>.............................] - ETA: 6:43:51 - loss: 1.3832 - accuracy: 0.2833

### Save the model

In [ ]:
model.save("capsnet_brain_tumor_model.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
model.save('capsnet_brain_tumor_model.keras')

### Evaluate the model

In [ ]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 2.5896432399749756
Test accuracy: 0.22883294522762299


### Generate classification report

In [ ]:
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_true, y_pred_classes, target_names=categories))

41/41 [==============================] - 7s 174ms/step
              precision    recall  f1-score   support

      glioma       0.23      1.00      0.37       300
  meningioma       0.00      0.00      0.00       306
     notumor       0.00      0.00      0.00       405
   pituitary       0.00      0.00      0.00       300

    accuracy                           0.23      1311
   macro avg       0.06      0.25      0.09      1311
weighted avg       0.05      0.23      0.09      1311



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Confusion matrix

In [ ]:
conf_matrix = confusion_matrix(y_true, y_pred_classes)
print(conf_matrix)

[[300   0   0   0]
 [306   0   0   0]
 [405   0   0   0]
 [300   0   0   0]]
